## Agreements

In [84]:
import os
import sys
import django
from tqdm.auto import tqdm

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

sys.path.append("..")
os.environ['DJANGO_SETTINGS_MODULE'] = 'news_labelling.settings'
django.setup()

from api.models import Comment, ArticleLabel, CommentLabel, Article
from django.contrib.auth.models import User

Primero, veamos qué usuarios etiquetaron más de 4 artículos...

In [85]:
from django.db.models import Count

users = []
for agg in ArticleLabel.objects.values('user__username').annotate(Count('user__username')):
    if agg['user__username__count'] >= 9:
         users.append(agg['user__username'])   
users

['jmperez', 'mkondra', 'ndebandi']

In [86]:
base_query = {
    "user__username__in": users
}
ArticleLabel.objects.filter(**base_query).count()

50

¿Cuántos etiquetamos todes?

In [87]:
import pandas as pd

pd.options.display.max_columns = 40


articles_labelled = {label.article_id for label in ArticleLabel.objects.filter(**base_query)}

df = pd.DataFrame({"name":users})
df.set_index("name", inplace=True)

for label in ArticleLabel.objects.filter(**base_query):
    username = label.user.username
    df.loc[username, label.article.title] = int(label.is_interesting)
    
df

,Condenan a ocho años de cárcel por corrupción al ex presidente ecuatoriano Rafael Correa,"Cecilia Roth: ""La marcha anticuarentena me pareció una irresponsabilidad sanitaria""",El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón,El proyecto del impuesto a la riqueza “está listo” y se presentará la próxima semana,Ante el cierre de los cines por la pandemia el presidente del INCAA quiere ponerle otro impuesto a Netflix,Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria”,"Sergio Berni: “Debemos ir a una cuarentena absoluta, mucho más rigurosa que al principio y sin transporte público”",Los piratas del siglo XXI se roban US$1000 millones por año del mar argentino,Juntos por el Cambio impulsa un proyecto en la provincia de Buenos Aires para sacarles los planes sociales a los que usurpen terrenos,¿Terminará Argentina como Venezuela?,"Al borde de las lágrimas, Luis Brandoni convocó un banderazo en contra del Gobierno, pero “cumpliendo todos los protocolos”",Coronavirus en la Argentina: Hebe de Bonafini pidió suspender la marcha del 24 de marzo,Intentó violar a su novia y ella lo asesinó a escobazos,Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios”,"Solo el 35,5% de la población aprueba la gestión de la pandemia realizada por el Gobierno",Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA para evitar contagios”,"Según un experto italiano, “el coronavirus se está debilitando y podría morir solo sin una vacuna”",Video: Jair Bolsonaro le hace upa a un hombre de talla baja pensando que era un niño,"Filminas para todos, menos para Maduro",Ginés González: “La semana que viene aviones de Aerolíneas irán a China a buscar insumos de bioseguridad”,Florencia Peña se diferenció de los argentinos que se quieren ir del país: “Muchos queremos seguir apostando”,Así fue el minuto a minuto del debate entre los candidatos a vicepresidente de Estados Unidos,Escándalo en España: fallan los tests de coronavirus fabricados en China,"""Evolucionen"": Romina Malaspina hizo un fuerte descargo tras los comentarios sexistas por su look en el noticiero","Luis Novaresio contó cómo será su boda con Braulio Bauab: ""Nos va a casar una rabina"""
name,,,,,,,,,,,,,,,,,,,,,,,,,
jmperez,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
mkondra,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ndebandi,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Busquemos los que están etiquetados por todos

In [88]:
df = df[df.columns[df.notna().all()]]

df

,Condenan a ocho años de cárcel por corrupción al ex presidente ecuatoriano Rafael Correa,"Cecilia Roth: ""La marcha anticuarentena me pareció una irresponsabilidad sanitaria""",El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón,El proyecto del impuesto a la riqueza “está listo” y se presentará la próxima semana,Ante el cierre de los cines por la pandemia el presidente del INCAA quiere ponerle otro impuesto a Netflix,Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria”,"Sergio Berni: “Debemos ir a una cuarentena absoluta, mucho más rigurosa que al principio y sin transporte público”",Los piratas del siglo XXI se roban US$1000 millones por año del mar argentino,Juntos por el Cambio impulsa un proyecto en la provincia de Buenos Aires para sacarles los planes sociales a los que usurpen terrenos,¿Terminará Argentina como Venezuela?
name,,,,,,,,,,
jmperez,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
mkondra,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
ndebandi,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0


In [89]:
coders.shape

(3, 10)

In [90]:
from nltk import agreement

coders = df.values

formatted_labels = []
for i in range(coders.shape[0]):
    # Acá las ponemos siguiendo el siguiente formato
    # [num etiquetador, num instancia, label]
    for j in range(coders.shape[1]):
        formatted_labels.append((i, j, coders[i, j]))
        

ratingtask = agreement.AnnotationTask(data=formatted_labels)

ratingtask.kappa()

0.4666666666666666

0.46, agreement moderado... Bueno, vamos viendo

## Agreements sobre odio

Ok, acá agarremos 

In [108]:
article_labels.count()

15

In [107]:
article_labels = ArticleLabel.objects.filter(
    article__title__in=df.columns[df.all()],
    user__username__in=users,
)

formatted_labels = []

df_comments = pd.DataFrame({"name": users})

df_comments.set_index("name", inplace=True)

for article_label in article_labels:
    username = article_label.user.username
    for comment_label in article_label.comment_labels.all():
        df_comments.loc[username, comment_label.comment_id] = comment_label.is_hateful
    
df_comments

,294781,294782,294783,294784,294785,294786,294787,294788,294789,294790,294791,294792,294793,294794,294795,294796,294797,294798,294799,294800,...,326163,326164,326165,326166,326167,326168,326169,326170,326171,326172,326173,326174,326175,326176,326177,326178,326179,326180,326181,326182
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
jmperez,False,False,False,False,False,False,False,False,False,True,False,False,True,True,False,True,True,True,False,False,...,False,False,False,False,True,False,True,True,False,False,True,False,False,False,True,False,False,False,False,False
mkondra,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False
ndebandi,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,...,True,False,False,False,False,False,True,True,False,False,False,False,False,False,True,False,False,False,False,False


In [115]:
from nltk import agreement

coders = df_comments.values

formatted_labels = []
for i in range(coders.shape[0]):
    # Acá las ponemos siguiendo el siguiente formato
    # [num etiquetador, num instancia, label]
    for j in range(coders.shape[1]):
        formatted_labels.append((i, j, coders[i, j]))
        

ratingtask = agreement.AnnotationTask(data=formatted_labels)

ratingtask.kappa()

0.49533386600897433

In [116]:

df_comments.sum(axis=1)

name
jmperez     86.0
mkondra     57.0
ndebandi    62.0
dtype: float64

In [128]:
agreed = ((df_comments.loc["jmperez"] == df_comments.loc["mkondra"]) &\
(df_comments.loc["ndebandi"] == df_comments.loc["mkondra"]))

not_agreed_comments = df_comments[df_comments.columns[~agreed]]

not_agreed_comments

,294790,294793,294796,294797,294798,294802,294803,294808,294810,294811,294813,294817,294819,294822,294825,294827,294828,294830,292567,292568,...,263394,263395,263399,263403,263412,263419,263423,263425,263426,263428,263430,326154,326155,326158,326160,326161,326163,326167,326169,326173
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
jmperez,True,True,True,True,True,True,True,True,True,False,True,True,True,False,True,False,True,True,False,True,...,False,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True
mkondra,False,False,True,True,False,False,True,False,False,False,True,False,True,True,False,True,False,False,True,True,...,False,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True
ndebandi,False,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False,False,False,True,False,...,True,False,False,False,False,False,True,False,False,False,False,False,True,True,False,False,True,False,True,False


In [132]:
for comment_id in not_agreed_comments.columns:
    comment = Comment.objects.get(id=comment_id)
    
    article = comment.article
    
    print("="*80)
    
    print(article.title)
    print(comment.text)
    
    print(not_agreed_comments[comment_id])

Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria"
@LANACION Estas aparecen cuando están cagadas de hambre y les pasan el sobre!!!!😬😡
name
jmperez      True
mkondra     False
ndebandi    False
Name: 294790, dtype: object
Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria"
@LANACION A mí me pareció una IRRESPONSABILIDAD TOTAL VOTAR A LOS KIRCHNER Y PERMITIR QUE LA ARGENTINA SEA MANOSEADA DE ESTA MANERA X SEGUNDA VEZ. UDS., MILITONTOS QUEDARÁN EN LA HISTORIA COMO LOS MAYORES TRAIDORES DE LA SOCIEDAD ARGENTINA. DIOS LOS PERDONE, PORQUE EL PUEBLO ARGENTINO, NO!
name
jmperez      True
mkondra     False
ndebandi    False
Name: 294793, dtype: object
Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria"
@LANACION Nadie la juzga por su adicción a las drogas. Que no venga a criticar las expresiones de voluntad.
name
jmperez      True
mkondra      True
ndebandi    False
Name: 294796, dtype: object
Cecil